In [207]:
%load_ext autoreload
%autoreload 2

from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt 
from glob import glob
import os

import torch
import torch.nn as nn 
import torch.nn.functional as F
from torchvision import transforms as T
from torchvision.transforms.functional import InterpolationMode
from tensorflow.keras.utils import to_categorical

#---------------------------------------
import custom_dataset
import utils_rs

#----------------------------------------------
import swin
import upper_net_mmseg
import models


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [208]:
# model swin
model_swin = swin.swin()

# model upernet
model_upernet = upper_net_mmseg.UPerHead(
    in_channels = model_swin.out_channels[1:],
    channels = model_swin.out_channels[2],
    in_index= (0,1,2,3),
    dropout_ratio=0.1,
    #norm_cfg= dict(type='SyncBN', requires_grad=True)
)


# model samrs 
swin_samrs = models.SamRS(model1=model_swin, model2=model_upernet)

In [209]:

ISAID_CLASSES = ('background', 'ship', 'store_tank', 'baseball_diamond',
               'tennis_court', 'basketball_court', 'Ground_Track_Field',
               'Bridge', 'Large_Vehicle', 'Small_Vehicle', 'Helicopter',
               'Swimming_pool', 'Roundabout', 'Soccer_ball_field', 'plane',
               'Harbor')

ISAID_PALETTE = {
    0: (0, 0, 0), 1: (0, 0, 63), 2: (0, 63, 63), 3: (0, 63, 0), 4: (0, 63, 127),
    5: (0, 63, 191), 6: (0, 63, 255), 7: (0, 127, 63), 8: (0, 127, 127),
    9: (0, 0, 127), 10: (0, 0, 191), 11: (0, 0, 255), 12: (0, 191, 127),
    13: (0, 127, 191), 14: (0, 127, 255), 15: (0, 100, 155)}


In [210]:
img_path = "/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/04.1024_imgs"
mask_path = "/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/08.1024_masks_categorized_imgs_all"

assert len(os.listdir(img_path)) == len(os.listdir(mask_path))

segdataset = custom_dataset.SegDataset(img_path, mask_path, 512)

['/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/04.1024_imgs/P0000_0_0.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/04.1024_imgs/P0000_0_1.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/04.1024_imgs/P0000_0_2.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/04.1024_imgs/P0000_1_0.png']
['/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/08.1024_masks_categorized_imgs_all/0.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/08.1024_masks_categorized_imgs_all/1.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/08.1024_masks_categorized_imgs_all/2.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/08.1024_masks_categorized_imgs_all/3.png']


In [211]:
a, b = segdataset.__getitem__(0)

In [212]:
dummy_ = torch.randn((1,3,512,512))

In [213]:
dummy_.shape

torch.Size([1, 3, 512, 512])

In [214]:
a_ = a.unsqueeze(0)
out_ = swin_samrs(a_)

In [215]:
out_[0].shape

torch.Size([18, 512, 512])

In [216]:
b.shape

torch.Size([16, 512, 512])

In [152]:
swin_samrs.semseghead_1

Sequential(
  (0): Dropout2d(p=0.1, inplace=False)
  (1): Conv2d(192, 18, kernel_size=(1, 1), stride=(1, 1))
)

In [77]:
model_swin.out_channels[2]

192

In [153]:
# #         self.semseghead_1 = nn.Sequential(
#                 nn.Dropout2d(0.1),
#                 nn.Conv2d(self.encoder.out_channels[2], classes1, kernel_size=1)
#             )

# fine tune layer initialize

n_classes = 16
swin_samrs.semseghead_1 = nn.Sequential(
                                    nn.Dropout2d(0.1),
                                    nn.Conv2d(model_swin.out_channels[2], n_classes, kernel_size=1)
                                    )

In [154]:
swin_samrs.semseghead_1

Sequential(
  (0): Dropout2d(p=0.1, inplace=False)
  (1): Conv2d(192, 16, kernel_size=(1, 1), stride=(1, 1))
)

In [155]:
out_ = swin_samrs(dummy_)
print(out_[0].shape)

torch.Size([16, 512, 512])


In [156]:
b.shape

torch.Size([16, 512, 512])

In [123]:
pred = out_[0].unsqueeze(0)

In [126]:
pred.shape

torch.Size([1, 16, 512, 512])

In [127]:
target = b.unsqueeze(0)

In [128]:
target.shape

torch.Size([1, 16, 512, 512])

In [129]:
import loss_
loss_fn = loss_.DiceLoss()


In [130]:
qwe = loss_fn(pred, target)

In [131]:
qwe

tensor(1.0717, grad_fn=<RsubBackward1>)

In [81]:
dummy_ = dummy_.to("cuda:0")

# load
w_path = "./swint_upernet_imp_sep_model.pth"
weights_ = torch.load(w_path, map_location=torch.device('cpu'))
swin_samrs.load_state_dict(  weights_['state_dict'] )

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│ /mnt/hdd/eric/.conda/envs/8.tmp.copied/lib/python3.8/site-packages/torch/nn/modules/module.py:16 │
│ 71 in load_state_dict                                                                            │
│                                                                                                  │
│   1668 │   │   │   │   │   │   ', '.join('"{}"'.format(k) for k in missing_keys)))               │
│   1669 │   │                                                                                     │
│   1670 │   │   if len(error_msgs) > 0:                                                           │
│ ❱ 1671 │   │   │   raise RuntimeError('Error(s) in loading state_dict for {}:\n\t{}'.format(     │
│   1672 │   │   │   │   │   │   │      self.__class__.__name__, "\n\t".join(error_msgs)))         │
│   1673 │   │   return _IncompatibleKeys(missing_keys, unexpected_keys)                           │
│   1674                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: Error(s) in loading state_dict for SamRS:
        Missing key(s) in state_dict: "decoder.psp_modules.0.1.conv.bias", "decoder.psp_modules.1.1.conv.bias", 
"decoder.psp_modules.2.1.conv.bias", "decoder.psp_modules.3.1.conv.bias", "decoder.bottleneck.conv.bias", 
"decoder.lateral_convs.0.conv.bias", "decoder.lateral_convs.1.conv.bias", "decoder.lateral_convs.2.conv.bias", 
"decoder.fpn_convs.0.conv.bias", "decoder.fpn_convs.1.conv.bias", "decoder.fpn_convs.2.conv.bias", 
"decoder.fpn_bottleneck.conv.bias", "decoder.semseghead_1.1.weight", "decoder.semseghead_1.1.bias". 
        Unexpected key(s) in state_dict: "decoder.psp_modules.0.1.bn.weight", "decoder.psp_modules.0.1.bn.bias", 
"decoder.psp_modules.0.1.bn.running_mean", "decoder.psp_modules.0.1.bn.running_var", 
"decoder.psp_modules.0.1.bn.num_batches_tracked", "decoder.psp_modules.1.1.bn.weight", 
"decoder.psp_modules.1.1.bn.bias", "decoder.psp_modules.1.1.bn.running_mean", 
"decoder.psp_modules.1.1.bn.running_var", "decoder.psp_modules.1.1.bn.num_batches_tracked", 
"decoder.psp_modules.2.1.bn.weight", "decoder.psp_modules.2.1.bn.bias", "decoder.psp_modules.2.1.bn.running_mean", 
"decoder.psp_modules.2.1.bn.running_var", "decoder.psp_modules.2.1.bn.num_batches_tracked", 
"decoder.psp_modules.3.1.bn.weight", "decoder.psp_modules.3.1.bn.bias", "decoder.psp_modules.3.1.bn.running_mean", 
"decoder.psp_modules.3.1.bn.running_var", "decoder.psp_modules.3.1.bn.num_batches_tracked", 
"decoder.bottleneck.bn.weight", "decoder.bottleneck.bn.bias", "decoder.bottleneck.bn.running_mean", 
"decoder.bottleneck.bn.running_var", "decoder.bottleneck.bn.num_batches_tracked", 
"decoder.lateral_convs.0.bn.weight", "decoder.lateral_convs.0.bn.bias", "decoder.lateral_convs.0.bn.running_mean", 
"decoder.lateral_convs.0.bn.running_var", "decoder.lateral_convs.0.bn.num_batches_tracked", 
"decoder.lateral_convs.1.bn.weight", "decoder.lateral_convs.1.bn.bias", "decoder.lateral_convs.1.bn.running_mean", 
"decoder.lateral_convs.1.bn.running_var", "decoder.lateral_convs.1.bn.num_batches_tracked", 
"decoder.lateral_convs.2.bn.weight", "decoder.lateral_convs.2.bn.bias", "decoder.lateral_convs.2.bn.running_mean", 
"decoder.lateral_convs.2.bn.running_var", "decoder.lateral_convs.2.bn.num_batches_tracked", 
"decoder.fpn_convs.0.bn.weight", "decoder.fpn_convs.0.bn.bias", "decoder.fpn_convs.0.bn.running_mean", 
"decoder.fpn_convs.0.bn.running_var", "decoder.fpn_convs.0.bn.num_batches_tracked", 
"decoder.fpn_convs.1.bn.weight", "decoder.fpn_convs.1.bn.bias", "decoder.fpn_convs.1.bn.running_mean", 
"decoder.fpn_co

In [82]:
swin_samrs = swin_samrs.to("cuda:0")

In [53]:
out_ = swin_samrs(dummy_)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ /mnt/hdd/eric/.conda/envs/8.tmp.copied/lib/python3.8/site-packages/torch/nn/modules/module.py:11 │
│ 94 in _call_impl                                                                                 │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/models.py:42 in forward            │
│                                                                                                  │
│   39 │                                                                                           │
│   40 │   def forward(self,x):                                                                    │
│   41 │   │   features = self.encoder(x)                                                          │
│ ❱ 42 │   │   output = self.decoder(*features)                                                    │
│   43 │   │   output_1 = self.semseghead_1(output)                                                │
│   44 │   │   output_2 = self.semseghead_2(output)                                                │
│   45 │   │   output_3 = self.semseghead_3(output)                                                │
│                                                                                                  │
│ /mnt/hdd/eric/.conda/envs/8.tmp.copied/lib/python3.8/site-packages/torch/nn/modules/module.py:11 │
│ 94 in _call_impl                                                                                 │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/upper_net_mmseg.py:566 in forward  │
│                                                                                                  │
│   563 │   │                                                                                      │
│   564 │   │   inputs = inputs[1:]                                                                │
│   565 │   │                                                

In [78]:
import loss_

In [79]:
diceloss = loss_.DiceLoss()

In [8]:
#-----------------------------------------

import wandb
import logging
from tqdm import tqdm
from lightning.fabric import Fabric
import lightning as L 

In [16]:
# # distribute gpus
# device_list = [0,1,2,3]
# fabric = L.Fabric(accelerator="cuda", devices=device_list, strategy='ddp')
# fabric.launch()

In [17]:
# log 
logging.basicConfig(filename='./1.log/model_v1.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


In [18]:
!wandb login

wandb: Currently logged in as: ericpark. Use `wandb login --relogin` to force relogin


In [19]:
# opt 
optimizer = torch.optim.AdamW(swin_samrs.parameters(), lr=1e-5)

# model 
swin_samrs = swin_samrs.train()
swin_samrs = swin_samrs.to("cuda:0")

# dataset 
batch_size= 4 
segdataset = custom_dataset.SegDataset(img_path, mask_path, 512)
dataloader = torch.utils.data.DataLoader(segdataset, batch_size=batch_size, shuffle=True)
#dataloader = fabric.setup_dataloaders(dataloader)

# Define loss
criterion = nn.CrossEntropyLoss()


# run 
epochs = 999
for epoch in range(epochs):

    iteration = 0
    running_loss = 0 
    
    for i, data in enumerate(tqdm(dataloader, desc=f"Epoch {epoch}")):

        img, mask = data

        img = img.to("cuda:0")
        mask = mask.to("cuda:0")
        
         
        # opt 
        optimizer.zero_grad()
        
        # run
        outputs = swin_samrs(img)
        
        # criterion
        
        loss = criterion(outputs, mask)
        #print(loss)
        
        loss.backward()
        #fabric.backward(loss)
        optimizer.step()

        # stat
        running_loss += loss.item()

        # log
        logger.info(f"[{epoch}, {i}] loss: {loss:.8f}")

        
        log = {'loss': f'{loss / 10:.8f}' }
        #print(log)
        wandb.log(log)

        log_iter = 200
        if (i % log_iter) == 0:    # print every 2000 mini-batches
            print(f"epoch : {epoch} iter : {i} /  total_iter : {len(dataloader)} running_loss : {running_loss / log_iter}")
            
            running_loss = 0.0
         
            
        #-----
    #-- epoch
    save_path = f"./2.ckpts/swin_rs_{epoch + 1}.pt"
    torch.save(swin_samrs.state_dict(), save_path)


['/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/04.1024_imgs/P0000_0_0.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/04.1024_imgs/P0000_0_1.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/04.1024_imgs/P0000_0_2.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/04.1024_imgs/P0000_1_0.png']
['/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/08.1024_masks_categorized_imgs_all/0.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/08.1024_masks_categorized_imgs_all/1.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/08.1024_masks_categorized_imgs_all/2.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/08.1024_masks_categorized_imgs_all/3.png']


Epoch 0:   0%|          | 0/21 [00:00<?, ?it/s]/mnt/hdd/eric/.conda/envs/8.tmp.copied/lib/python3.8/site-packages/torchvision/transforms/functional.py:149: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:199.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()
Epoch 0:   0%|          | 0/21 [00:01<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:37                                                                                   │
│                                                                                                  │
│ /mnt/hdd/eric/.conda/envs/8.tmp.copied/lib/python3.8/site-packages/torch/nn/modules/module.py:11 │
│ 94 in _call_impl                                                                                 │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/models.py:42 in forward            │
│                                                                                                  │
│   39 │                                                                                           │
│   40 │   def forward(self,x):                                                                    │
│   41 │   │   features = self.encoder(x)                                                          │
│ ❱ 42 │   │   output = self.decoder(*features)                                                    │
│   43 │   │   output_1 = self.semseghead_1(output)                                                │
│   44 │   │   output_2 = self.semseghead_2(output)                                                │
│   45 │   │   output_3 = self.semseghead_3(output)                                                │
│                                                                                                  │
│ /mnt/hdd/eric/.conda/envs/8.tmp.copied/lib/python3.8/site-packages/torch/nn/modules/module.py:11 │
│ 94 in _call_impl                                                                                 │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/upper_net_mmseg.py:566 in forward  │
│                                                                                                  │
│   563 │   │                                                                                      │
│   564 │   │   inputs = inputs[1:]                                                                │
│   565 │   │                                                